# Get Tweets in Real Time using Twitter API

We are using Tweepy library to route all the Live tweets to a TCP socket server.<br> 
This server is hosted in Local Machine.<br>
Import tweepy library<br>
Import OAuthHandler.<br> It is used for Authentication protocol (OAuth) of Twitter API.<br>
Import Stream & StreamListner.<br> This actually listen to streaming data getting from the socket.<br>
Import socket. It is used to create socket connection with Localhost i.e. local server. <br>
Json is used because twitter data comes in json format.<br>


In [ ]:
import os
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

To connect to Twitter API, we need these 4 Keys. 
It can be generated from https://apps.twitter.com/ by creating a new app.

In [ ]:
consumer_key    = 't0O1DuEyMcDpFsJRLYzAXfIZd'
consumer_secret = 'trcnjqVLPp1LNXqBcAHjA1M5RLorlNnpDxKWDrNCVhW82aktuR'
access_token    = '2341567088-0n5AEvesiUL34b0vKZUWJ91MJkNFAHFnGMxfRUo'
access_secret   = 'SkaxZb0dRD0lixXNcGdTCVcTXaasZ13OkQHkhnD5VhUI1'

#### Class TweetsListner
    1.It is listening to the stream via StreamListener. 
    2.constrctor function (init) : initializing the socket. 
    3.data contains the tweets json coming from the stream. 
    4.In this json object field 'text' contains the actual Tweet.
    5.The actual tweet is extracted & sent to the client socket.
    6.Error handling is done to catch & throw errors.

In [ ]:
class TweetsListener(StreamListener):
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:
            msg = json.loads( data )
            print(msg['user']['screen_name'].encode('utf-8'))
            print( msg['text'].encode('utf-8') )
            print( msg['created_at'].encode('utf-8') )
            self.client_socket.send( msg['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    def on_error(self, status):
        print(status)
        return True

### Function SendData():
    1. auth is doing Authentication check using OAuthHandler. It uses 4 keys of Twitter APIs.
    2. consumer_key & consumer_secret are like username & access_token & access_secret are like password.
    3. twitter_stream will get the actual twitter live stream data.
    4. It'll call Stream with auth & call the Class TweetsListner & pass c_socket info to the class.
    5. From the stream of tweets, sample() method helps in getting whole corpus of tweets.

In [ ]:
def sendData(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.sample()
    #twitter_stream.filter(track=['india'])

### Socket Creation:
    1. Socket is created by using socket function.
    2. Host is localhost, which in our case is IP of Cloudxlab, where this Jupyter notebook is running.
    3. Port is 6782 (It can be anything, unique for this service)
    4. Now bind host & port address to the socket.

In [ ]:
s = socket.socket()     # Create a socket object
host = "127.0.0.1"      # Get local machine name
port = 6782            # Reserve a port for your service.
s.bind((host, port))    # Bind to the port
print("Listening on port: %s" % str(port))

socket will wait & listen for few seconds. <br>
After that connection is made.

In [ ]:
s.listen(5)                 # Now wait for client connection.
c, addr = s.accept()        # Establish connection with client.
print( "Received request from: " + str( addr ) )

### Send live Tweets in real time to the Socket
Function sendData will send all the Tweets  to socket 'c' in real time.<br>
This will be consumed in the next code of Spark Streaming to do Trending topic Analysis.

In [ ]:
sendData( c )